In [18]:
import yaml
with open("../plotly.yml") as stream:
    plt_yml = yaml.load(stream)

In [19]:
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
import pandas as pd

# setting user, api key and access token
plotly.tools.set_credentials_file(username=plt_yml["username"], api_key=plt_yml["api_key"])
mapbox_access_token = plt_yml["mapbox_access_token"]


In [20]:
df = pd.read_csv("../data/PPR-dublin-lt_1m.csv")

In [21]:
df.head()

,Unnamed: 0,Date of Sale (dd/mm/yyyy),Address,Postal Code,County,Price (€),Not Full Market Price,VAT Exclusive,Description of Property,Property Size Description,geocode,lat,lon,class,importance,country,price
0,0,01/01/2010,"5 Braemor Drive, Churchtown, Co.Dublin",NaN,Dublin,"€343,000.00",No,No,Second-Hand Dwelling house /Apartment,NaN,"{'place_id': 172843344, 'licence': 'Data © Ope...",53.302406,-6.263796,building,0.611,Ireland,343000.0
1,2,04/01/2010,"1 Meadow Avenue, Dundrum, Dublin 14",NaN,Dublin,"€438,500.00",No,No,Second-Hand Dwelling house /Apartment,NaN,"{'place_id': 79202326, 'licence': 'Data © Open...",53.286736,-6.254219,highway,0.700,Ireland,438500.0
2,5,04/01/2010,"12 Sallymount Avenue, Ranelagh",NaN,Dublin,"€425,000.00",No,No,Second-Hand Dwelling house /Apartment,NaN,"{'place_id': 241009782, 'licence': 'Data © Ope...",53.324710,-6.253631,highway,0.320,Ireland,425000.0
3,11,04/01/2010,"206 Philipsburgh Avenue, Marino, Dublin 3",NaN,Dublin,"€430,000.00",No,No,Second-Hand Dwelling house /Apartment,NaN,"{'place_id': 157106166, 'licence': 'Data © Ope...",53.371258,-6.238079,building,0.521,Ireland,430000.0
4,12,04/01/2010,"22 Laverna Way, Castleknock, Dublin 15",NaN,Dublin,"€355,000.00",No,No,Second-Hand Dwelling house /Apartment,NaN,"{'place_id': 250288139, 'licence': 'Data © Ope...",53.373665,-6.383456,building,0.611,Ireland,355000.0


In [22]:
df["year"] = df["Date of Sale (dd/mm/yyyy)"].str.split("/").str.get(-1)

In [23]:
data = []
for event in df["year"].drop_duplicates():
    event_data = dict(
            lat = df.loc[df['year'] == event,'lat'],
            lon = df.loc[df['year'] == event,'lon'],
            name = event,
            text = df.loc[df['year'] == event,'price'].astype(str),
            marker = dict(size = 8, opacity = 0.5, color = df.loc[df['year'] == event,'price'], colorscale='Viridis'),
            type = 'scattermapbox'
        )
    data.append(event_data)

In [24]:
layout = dict(
    height = 800,
    # top, bottom, left and right margins
    margin = dict(t = 0, b = 0, l = 0, r = 0),
    font = dict(color = '#FFFFFF', size = 11),
    paper_bgcolor = '#000000',
    mapbox = dict(
        # here you need the token from Mapbox
        accesstoken = mapbox_access_token,
        bearing = 0,
        # where we want the map to be centered
        center = dict(
            lat = 53.35,
            lon = -6.25
        ),
        # we want the map to be "parallel" to our screen, with no angle
        pitch = 0,
        # default level of zoom
        zoom = 9,
        # default map style
        style = 'dark'
    )
)

In [25]:
years = df["year"].drop_duplicates().sort_values().values.tolist()
controller = [dict(label = str(x),
     method = 'update',
     args = [{'visible': [False if y!=i else True for y in range(len(years))]}])
              for i,x in enumerate(years)]
controller = [dict(label = "All",
     method = 'update',
     args = [{'visible': [True for y in range(len(years))]}])] + controller

In [26]:

updatemenus=list([
    # drop-down 1: map styles menu
    # buttons containes as many dictionaries as many alternative map styles I want to offer
    dict(
        buttons=list([
            dict(
                args=['mapbox.style', 'dark'],
                label='Dark',
                method='relayout'
            ),                    
            dict(
                args=['mapbox.style', 'light'],
                label='Light',
                method='relayout'
            ),
            dict(
                args=['mapbox.style', 'outdoors'],
                label='Outdoors',
                method='relayout'
            ),
            dict(
                args=['mapbox.style', 'satellite-streets'],
                label='Satellite with Streets',
                method='relayout'
            )                    
        ]),
        # direction where I want the menu to expand when I click on it
        direction = 'up',
      
        # here I specify where I want to place this drop-down on the map
        x = 0.75,
        xanchor = 'left',
        y = 0.05,
        yanchor = 'bottom',
      
        # specify font size and colors
        bgcolor = '#000000',
        bordercolor = '#FFFFFF',
        font = dict(size=11)
    ),    
    
    # drop-down 2: select type of storm event to visualize
    dict(
         # for each button I specify which dictionaries of my data list I want to visualize. Remember I have 7 different
         # types of storms but I have 8 options: the first will show all of them, while from the second to the last option, only
         # one type at the time will be shown on the map
         buttons=controller,
        # direction where the drop-down expands when opened
        direction = 'down',
        # positional arguments
        x = 0.01,
        xanchor = 'left',
        y = 0.99,
        yanchor = 'bottom',
        # fonts and border
        bgcolor = '#000000',
        bordercolor = '#FFFFFF',
        font = dict(size=11)
    )
])

# assign the list of dictionaries to the layout dictionary
layout['updatemenus'] = updatemenus


In [27]:
figure = dict(data = data, layout = layout)
py.iplot(figure, filename = 'dublin-ppr')